FAIRE TOURNER LE NOTEBOOK LOCATION_DATA AVANT CELUI-CI

In [1]:
%run location_data.ipynb

Carte non interactive de la rentabilité des départements

In [ ]:
def carte_rentabilite (df_loyer, df_achat, geo_dept):

    loyers_moyens_par_dept = df_loyer.groupby('code_dept')["Loyer d'annonce"].mean()
    df_dept = loyers_moyens_par_dept.reset_index()
    df_dept.columns = ['Numéro de département', 'Loyer moyen']

    prix_moyens_par_dept = df_achat.groupby('code_dept')["Prixm2Moyen"].mean()
            
    df_dept['Prix moyen'] = df_dept['Numéro de département'].map(prix_moyens_par_dept)

    df_dept['Rentabilité'] = ((df_dept['Loyer moyen']*12)/df_dept['Prix moyen'])*100

    ##Filtre des outliers car quelques valeurs aberantes

    Q1 = df_dept['Rentabilité'].quantile(0.25)
    Q3 = df_dept['Rentabilité'].quantile(0.75)
    IQR = Q3 - Q1
    lim_inf = Q1 - 1.5 * IQR
    lim_sup = Q3 + 1.5 * IQR

    df_dept_rentabilite = df_dept[(df_dept['Rentabilité'] >= lim_inf) & (df_dept['Rentabilité'] <= lim_sup)]

    df_top5 = df_dept_rentabilite.sort_values(by='Rentabilité', ascending=False).head(5)
    
    df_top5 = df_top5.merge(
        geo_dept[['code', 'nom']], 
        left_on='Numéro de département', 
        right_on='code', 
        how='left'
    )
    
    legend_items = ["Top 5 Rentabilité Brute:"]
    df_top5_reset = df_top5.reset_index() 
    for index, row in df_top5_reset.iterrows():
        item = f"{index + 1}. {row['nom']} ({row['Numéro de département']}) : {row['Rentabilité']:.2f}%"
        legend_items.append(item)
    legend_text = "\n".join(legend_items)

    carte_df = geo_dept.merge(
        df_dept_rentabilite, 
        left_on='code', 
        right_on='Numéro de département', 
        how='left'
    )

    carte_df['Loyer moyen'] = pd.to_numeric(carte_df['Loyer moyen'])
            
    carte_df['Rentabilité'] = carte_df['Rentabilité'].replace([np.inf, -np.inf], np.nan)

    fig, ax = plt.subplots(1, 1, figsize=(10, 10))

    carte_df.plot(
        ax=ax,
        color="lightgrey",
        edgecolor="0.8",
        linewidth=0.5
    )

    carte_avec_donnees = carte_df.dropna(subset=['Rentabilité'])

    carte_avec_donnees.plot(
        column='Rentabilité', 
        cmap='Reds',           
        linewidth=0.5,         
        ax=ax,                 
        edgecolor='0.8',     
        legend=True,
        legend_kwds={
            'label': "Rentabilité brute de la location (%)", # C'est 'title' qu'il faut ici
            'orientation': "horizontal",
            'shrink': 0.7 
        }
    )
            
    ax.set_axis_off() 
    ax.set_title(
        "Rentabilité brute moyenne par département en France", 
        fontdict={'fontsize': '16', 'fontweight' : '3'}
    )

    fig.text(0.25, 0.25, # Coordonnées (x, y) relatives à la figure (0,0 = bas gauche)
            legend_text,
            fontsize=10,
            bbox=dict(boxstyle='round,pad=0.5', fc='white', alpha=0.7),
            verticalalignment='bottom',
            horizontalalignment='left')

    plt.show()

    return df_dept_rentabilite

Carte interactive de la rentabilités des départements

In [ ]:
def carte_rentabilite_interactive(df_loyer, df_achat, geo_dept):
    """
    Calcule la rentabilité et génère une carte INTERACTIVE FIXE 
    avec un fond uni et une légende Top 5.
    """

    # --- 1. Préparation des données (INCHANGÉ) ---
    loyers_moyens_par_dept = df_loyer.groupby('code_dept')["Loyer d'annonce"].mean()
    df_dept = loyers_moyens_par_dept.reset_index()
    df_dept.columns = ['Numéro de département', 'Loyer moyen']

    prix_moyens_par_dept = df_achat.groupby('code_dept')["Prixm2Moyen"].mean()
    df_dept['Prix moyen'] = df_dept['Numéro de département'].map(prix_moyens_par_dept)

    df_dept['Rentabilité'] = ((df_dept['Loyer moyen'] * 12) / df_dept['Prix moyen']) * 100
    df_dept['Rentabilité'] = df_dept['Rentabilité'].replace([np.inf, -np.inf], np.nan)

    # --- 2. Filtre des outliers (INCHANGÉ) ---
    df_rent_valide = df_dept.dropna(subset=['Rentabilité'])
    Q1 = df_rent_valide['Rentabilité'].quantile(0.25)
    Q3 = df_rent_valide['Rentabilité'].quantile(0.75)
    IQR = Q3 - Q1
    lim_inf = Q1 - 1.5 * IQR
    lim_sup = Q3 + 1.5 * IQR

    df_dept_rentabilite = df_dept[
        (df_dept['Rentabilité'] >= lim_inf) & (df_dept['Rentabilité'] <= lim_sup)
    ]

    # --- 3. Préparation du Top 5 (INCHANGÉ) ---
    df_top5 = df_dept_rentabilite.sort_values(by='Rentabilité', ascending=False).head(5)
    df_top5 = df_top5.merge(
        geo_dept[['code', 'nom']],
        left_on='Numéro de département',
        right_on='code',
        how='left'
    )
    
    # Préparation du texte pour la légende (sera utilisé plus bas)
    legend_items_html = ["<b>Top 5 Rentabilité Brute :</b>"]
    df_top5_reset = df_top5.reset_index()
    for index, row in df_top5_reset.iterrows():
        item = f"{index + 1}. {row['nom']} ({row['Numéro de département']}) : {row['Rentabilité']:.2f}%"
        legend_items_html.append(item)
    legend_html_content = "<br>".join(legend_items_html)
    
    # On garde aussi la version markdown pour le retour, au cas où
    legend_text_markdown = "\n".join(legend_items_html).replace('<b>', '**').replace('</b>', '**')


    # --- 4. Jointure finale pour la carte (INCHANGÉ) ---
    carte_df = geo_dept.merge(
        df_dept_rentabilite,
        left_on='code',
        right_on='Numéro de département',
        how='left'
    )
    
    carte_df['Loyer moyen'] = pd.to_numeric(carte_df['Loyer moyen'])
    carte_df['Rentabilité'] = carte_df['Rentabilité'].round(2)
    carte_df['Loyer moyen'] = carte_df['Loyer moyen'].round(2)
    carte_df['Prix moyen'] = carte_df['Prix moyen'].round(2)


    # --- 5. GÉNERATION DE LA CARTE (PARTIE MODIFIÉE) ---
    
    # On définit le centre et les limites pour la France
    center_france = [46.6, 2.2]
    initial_zoom = 6
    # Limites (Sud-Ouest, Nord-Est) pour bloquer la carte
    bounds = [[41, -5.5], [51.2, 9.8]]

    m = carte_df.explore(
        column='Rentabilité',
        cmap='Reds',
        tooltip=['nom', 'Numéro de département', 'Rentabilité', 'Loyer moyen', 'Prix moyen'],
        legend=True,
        legend_kwds={'caption': "Rentabilité brute de la location (%)"},
        missing_kwds={"color": "lightgrey", "label": "Données manquantes / Outliers"},
        style_kwds={'stroke': True, 'color': 'black', 'weight': 0.5},
        highlight_kwds={'fillOpacity': 0.7, 'weight': 2, 'color': 'white'},
        
        # --- NOUVELLES OPTIONS ---
        tiles=None, # Supprime la carte du monde en arrière-plan
        
        # Options passées à folium.Map pour désactiver les interactions
        m_kwgs={
            'location': center_france,
            'zoom_start': initial_zoom,
            'dragging': False,
            'zoom_control': False,
            'scrollWheelZoom': False,
            'doubleClickZoom': False
        }
    )

    # --- AJOUTS POST-EXPLORE POUR FIXER LA CARTE ET AJOUTER LA LÉGENDE ---
    
    # 1. Bloquer la carte sur la France
    m.fit_bounds(bounds)
    m.options['maxBounds'] = bounds
    m.options['minZoom'] = initial_zoom
    m.options['maxZoom'] = initial_zoom # Bloque le zoom au niveau initial

    # 2. Ajouter le Top 5 comme un bloc HTML fixe sur la carte
    # (Similaire à ce que vous aviez sur l'image statique)
    html_legend_box = f"""
    <div style="position: fixed;
                bottom: 50px; left: 20px; width: 280px;
                background-color: rgba(255, 255, 255, 0.8);
                border: 1px solid grey; border-radius: 5px;
                z-index:9999; font-size:13px; padding: 10px;
                font-family: Arial, sans-serif;">
        {legend_html_content}
    </div>
    """
    m.get_root().html.add_child(Element(html_legend_box))

    # La fonction retourne la carte ET le texte (même si le texte est déjà sur la carte)
    return m, legend_text_markdown, df_dept_rentabilite

Trouver le loyer optimal pour un bien. On se base sur l'écart relatif des données entre les moyennes du département et les valeurs du bien.

In [ ]:
def calcul_rentabilite (prix_achat, loyer, nb_m2, dept, df_dept_rentabilite):
    
    if nb_m2 <= 0 or prix_achat <= 0:
        print("Erreur : Le prix d'achat et les m2 doivent être supérieurs à 0.")
        return
    
    if dept == "":
        print("Entrez un numéro de département")
        return
    elif dept.isdigit() != True:
        print("Entrez un nombre")
        return
    elif int(dept) <1 or int(dept) >95 or len(dept) !=2:
        print("Entrez un département valide")
        return
    
    prix_achat_m2 = prix_achat/nb_m2
    loyer_m2 = loyer/nb_m2
    rentabilite = ((loyer_m2*12)/prix_achat_m2)*100

    ligne_dept = df_dept_rentabilite[df_dept_rentabilite['Numéro de département'] == dept]
    renta_dept = ligne_dept['Rentabilité'].iloc[0]
    loyer_dept = ligne_dept['Loyer moyen'].iloc[0]

    ecart_relatif_rentabilite = ((rentabilite/renta_dept) - 1) * 100
    ecart_relatif_loyer = ((loyer_dept/loyer_m2)-1) * 100

    #figure
    categories = ['Rentabilité', 'Loyer']
    valeurs = [ecart_relatif_rentabilite, ecart_relatif_loyer]
    couleurs = []

    for v in valeurs:
        if v > 15:
            couleurs.append('green')
        elif v < -15:
            couleurs.append('red')
        else: 
            couleurs.append('orange') 

    fig, ax= plt.subplots()
    ax.barh(categories, valeurs, color=couleurs)

    for index, valeur in enumerate(valeurs):
        ax.text(valeur + 1, index, f"{valeur:+.0f}%", va='center', color='black')
    
    ax.axvline(0, color='black', linewidth=1.2)
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.spines['bottom'].set_visible(False)
    ax.spines['left'].set_visible(False)
    ax.get_xaxis().set_ticks([]) # Cache l'axe X
    ax.grid(axis='x', linestyle='--', alpha=0.7)
    plt.title(f"Positionnement vs Moyenne du Dpt: {dept} (Rentabilité Dpt: {renta_dept:.2f}%, Loyer Dpt: {loyer_dept:.2f}€/m², Rentabilité Bien: {rentabilite:.2f}%, Loyer Bien: {loyer_m2:.2f}€/m²)")
    plt.show()